In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import urllib.request
import time
import pymongo
from splinter import Browser
import pandas as pd

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## NASA Mars News:

In [4]:
#NASA Mars News Website
news_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(news_url)

In [5]:
#Parse HTML
news_html = browser.html
soup = BeautifulSoup(news_html, 'html.parser')

In [8]:
#News Title
news_title = soup.find('div', class_='bottom_gradient').text
#News Teaser Senter
news_teaser = soup.find('div', class_='article_teaser_body').text

print(news_title)
print(news_teaser)


Small Satellite Mission of the Year
The first interplanetary CubeSats were recognized by the engineering community with the 2019 Small Satellite Mission of the Year award.


## JPL Mars Space Images - Featured Image:

In [33]:
#JPL Mars Space Image website
mars_img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(mars_img_url)

In [29]:
#Set up Main URL
jpl_url = 'https://www.jpl.nasa.gov'

In [30]:
#Parse HTML
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [50]:
results = soup.find('article')['style']
print(results)

background-image: url('/spaceimages/images/wallpaper/PIA19046-1920x1200.jpg');


In [51]:
mars_image = results.replace('background-image: url(','').replace( ');', '')[1:-1]
mars_image

'/spaceimages/images/wallpaper/PIA19046-1920x1200.jpg'

In [53]:
featured_url = jpl_url + mars_image
print(featured_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19046-1920x1200.jpg


## Mars Weather:

In [14]:
# Twitter with Mars weather updates
weather_url = 'https://twitter.com/marswxreport?lang=en'

# Retrieve page with the requests module
response = requests.get(weather_url)
# Parse HTML
soup = BeautifulSoup(response.text, 'html.parser')

In [15]:
#Retrieve most recent weather facts
mars_weather = soup.find('p', class_='TweetTextSize').text
print(mars_weather)

InSight sol 249 (2019-08-09) low -98.8ºC (-145.8ºF) high -26.0ºC (-14.8ºF)
winds from the SSE at 4.4 m/s (9.8 mph) gusting to 17.5 m/s (39.1 mph)
pressure at 7.60 hPapic.twitter.com/jDOsvHTwYg


## Mars Facts:

In [18]:
#URL with Mars Facts Table
facts_url = 'https://space-facts.com/mars/'

In [21]:
tables = pd.read_html(facts_url)
tables

[  Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [22]:
#Creat Dataframe of Mars Table
df = tables[1]

#Map Column Names
mars_df = df[[0, 1]]
mapping = {0: "Description", 
                  1: "Value"}
mars_df = mars_df.rename(columns=mapping)
mars_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [23]:
#Save Dataframe as HTML
mars_df.to_html('mars_fact_table.html', index = None)

## Mars Hemispheres:

In [25]:
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#Set up main URL
usgs_url = 'https://astrogeology.usgs.gov'

#Locate all images
images = soup.find_all('div', class_='item')

#Creat Lists to store dictionary of image titles and image urls
img_info = []

for image in images:
    # Store title     
    img_title = image.find('h3').text 
    
    #For each image, pull Partial URL for Image 
    #partial href
    partial_href = image.find('a')['href']

    #Set up Image URL to visit
    browser.visit(usgs_url + partial_href)
    visit_img_html = browser.html

    # Parse HTML and retrieve full image source  
    soup = BeautifulSoup(visit_img_html, 'html.parser')
    img_url = usgs_url + soup.find('img', class_='wide-image')['src']

    # Create Dictionary from lists
    img_info.append({'Title': img_title, 'Image_URL': img_url})

img_info

[{'Title': 'Cerberus Hemisphere Enhanced',
  'Image_URL': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'Title': 'Schiaparelli Hemisphere Enhanced',
  'Image_URL': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'Title': 'Syrtis Major Hemisphere Enhanced',
  'Image_URL': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'Title': 'Valles Marineris Hemisphere Enhanced',
  'Image_URL': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]